In [ ]:
import osmnx as ox
import os 
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qtagg import FigureCanvasQTAgg as FigureCanvas, NavigationToolbar2QT as NavigationToolbar
from PyQt6.QtWidgets import QApplication, QDialog, QVBoxLayout, QLineEdit, QPushButton, QCheckBox, QLabel, QMessageBox, QDialogButtonBox
ox.__cached__ = True

In [ ]:
app = QApplication(os.sys.argv)

In [ ]:
class InputDialog(QDialog):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("City Road Network Mapper via OSM")
        self.setGeometry(100, 100, 300, 100)

        layout = QVBoxLayout()

        layout.addWidget(QLabel("Which city do you want to view? (City, State (if any) , Country in full)"))
        self.text_input = QLineEdit()
        layout.addWidget(self.text_input)
        
        layout.addWidget(QLabel("Select the type of Network you want to print"))
        self.checkbox_group = []
        choices = ["motorway","trunk","primary","secondary","tertiary","motorway_link","trunk_link"]
        for choice in choices:
            checkbox = QCheckBox(choice)
            self.checkbox_group.append(checkbox)
            layout.addWidget(checkbox)

        layout.addWidget(QLabel("Road Network Color? HEX with # (Default: White #fff)"))
        self.color_hex = QLineEdit()
        
        layout.addWidget(self.color_hex)

        layout.addWidget(QLabel("Background Color? HEX with # (Default: Black #000)"))
        self.bgcolor_hex = QLineEdit()
        
        layout.addWidget(self.bgcolor_hex)

        submit_button = QPushButton("Submit")
        submit_button.clicked.connect(self.submit_clicked)
        layout.addWidget(submit_button)

        self.setLayout(layout)

    def submit_clicked(self):
        try:
            self.geocode_city()
            selected_choices = [checkbox.text() for checkbox in self.checkbox_group if checkbox.isChecked()]

            formatted_choices = "|".join(selected_choices)
            self.selected_choices_str = f'["highway"~"{formatted_choices}"]'

            # Create the graph using OSMnx
            G = ox.graph_from_polygon(self.geocoded_city.geometry.to_list()[0], custom_filter=self.selected_choices_str)
            nodes, edges = ox.graph_to_gdfs(G)

            # Create the figure with fixed size
            f, ax = plt.subplots(1, 1, figsize=(10, 10))  # Set the size to 10x10 inches
            ax.set_aspect('equal')  # Ensure the aspect ratio is equal

            if not self.color_hex:
                self.color_hex = '#fff'
            if not self.bgcolor_hex:
                self.bgcolor_hex = '#000'
            edges.plot(ax=ax, color=self.color_hex.text())
            ax.set_facecolor(self.bgcolor_hex.text())

            # Get rid of the ticks
            ax.set_xticks([])
            ax.set_yticks([])

            # Add the title
            ymin, ymax = ax.get_ylim()
            extension = 0.2 * (ymax - ymin)
            ax.set_ylim(ymin, ymax + extension)
            ax.set_title(self.text_input.text(), fontsize=30, color=self.color_hex.text(), y=0.9)

            # Create a canvas to display the figure
            canvas = FigureCanvas(f)

            # Create and display the custom dialog
            self.show_canvas_in_dialog(canvas)
        except Exception as e:
            print(f"An error occurred: {e}")
            QMessageBox.critical(self, "Error", f"An error occurred: {e}")

    def geocode_city(self):
        self.geocoded_city = ox.geocode_to_gdf(self.text_input.text())

    def show_canvas_in_dialog(self, canvas):
        dialog = QDialog(self)
        dialog.setWindowTitle("Network Map")
        dialog.setGeometry(150, 150, 800, 800)

        layout = QVBoxLayout()

        # Add the canvas and toolbar to the layout
        layout.addWidget(canvas)
        toolbar = NavigationToolbar(canvas, dialog)
        layout.addWidget(toolbar)

        # Add Refresh button to update the plot
        refresh_button = QPushButton("Refresh Data")
        refresh_button.clicked.connect(self.refresh_plot)
        layout.addWidget(refresh_button)

        # Add OK button to close the dialog
        button_box = QDialogButtonBox(QDialogButtonBox.StandardButton.Ok)
        button_box.accepted.connect(dialog.accept)
        layout.addWidget(button_box)

        dialog.setLayout(layout)
        dialog.exec()

    def refresh_plot(self):
        try:
            self.geocode_city()
            selected_choices = [checkbox.text() for checkbox in self.checkbox_group if checkbox.isChecked()]

            formatted_choices = "|".join(selected_choices)
            self.selected_choices_str = f'["highway"~"{formatted_choices}"]'

            # Create the graph using OSMnx
            G = ox.graph_from_polygon(self.geocoded_city.geometry.to_list()[0], custom_filter=self.selected_choices_str)
            nodes, edges = ox.graph_to_gdfs(G)

            # Create the figure with fixed size
            f, ax = plt.subplots(1, 1, figsize=(10, 10))  # Set the size to 10x10 inches
            ax.set_aspect('equal')  # Ensure the aspect ratio is equal

            if not self.color_hex:
                self.color_hex = '#fff'
            if not self.bgcolor_hex:
                self.bgcolor_hex = '#000'
            edges.plot(ax=ax, color=self.color_hex.text())
            ax.set_facecolor(self.bgcolor_hex.text())

            # Get rid of the ticks
            ax.set_xticks([])
            ax.set_yticks([])

            # Add the title
            ymin, ymax = ax.get_ylim()
            extension = 0.2 * (ymax - ymin)
            ax.set_ylim(ymin, ymax + extension)
            ax.set_title(self.text_input.text(), fontsize=30, color=self.color_hex.text(), y=0.9)

            # Update the canvas with the new figure
            canvas = FigureCanvas(f)
            self.show_canvas_in_dialog(canvas)
        except Exception as e:
            print(f"An error occurred: {e}")
            QMessageBox.critical(self, "Error", f"An error occurred: {e}")

dialog = InputDialog()
dialog.exec()
